In [1]:
import asyncio
import json
import logging
import websockets
import time

logging.basicConfig()

usersWithoutID = set() # Array für User ohne zugeordnete ID

usersWithID = set() # Array für User mit ID

# Klasse für eine Nachricht
class message:
    session_id = 0
    type = ''
    message = ''
    send_time = int(round(time.time() * 1000)) # Timestamp in Millisekunden
    def __init__(self, id, type, message):
        self.session_id = id
        self.type = type
        self.message = message

# Klasse für einen User
class user:
    session_id = 0
    canControl = False
    def __init__(self, id, control, websocket):
        self.session_id = id
        self.canControl = control
        self.websocket = websocket


In [2]:
# Message an alle User der Session senden
async def sendMessageToSession(sessionID, message):
    if usersWithID:
        for user in usersWithID:
            if (user.session_id == sessionID):
                ws = user.websocket
                await asyncio.wait(ws.send(message))
            
# Einem Nutzer eine Nachricht übermitteln
async def sendMessageToUser(websocket, message):
    await asyncio.wait(websocket.send(message));

# Input an Session senden
async def sendNewInput(sessionID, inputType):
    message = message(sessionID, 'input', inputType)
    sendMessageToSession(sessionID, message)

# Überprüfe, ob es in der Session schon eine Person mit Steuerrechten gibt
def getContolStatus(sessionID):
    controller = [user for user in usersWithID if (user.session_id == sessionID and user.canControl == True)]
    
    if(controller == None):
        return True # Return Steuerrechte für Nutzer
    else:
        return False # Keine Rechte

# Steuerrechte des Nutzers aktualisieren
def changeControlStatus(sessionID, websocket):
    for index, user in enumerate(usersWithID):
        if(user.websocket == websocket and user.sessionID == sessionID):
            tmpUser = user(sessionID, websocket, True)
            user = replace_all(user, tmpUser)
            usersWithID[index] = user

# Registriere neuen Nutzer ohne ID
async def registerUserWithoutID(websocket):
    usersWithoutID.add(websocket)
            
# Registriere neuen Nutzer mit ID
async def registerUserWithID(websocket, sessionID):
    canControl = getControlStatus(sessionID) # Steuerstatus holen
    newUser = user(sessionID, canControl, websocket)
    usersWithID.add(newUser)
    unregisterUserWithoutID(websocket)
    # Registrierungsnachricht an User mit Steuerrechten
    if(canControl):
        message = message(sessionID, 'registered control', 'Nutzer wurde mit Steuerrechten registriert')
    else:
        message = message(sessionID, 'registered noControl', 'Nutzer wurde ohne Steuerrechten registriert')
        
    sendMessageToUser(websocket, message)
    
# Entferne Nutzer ohne ID
async def unregisterUserWithoutID(websocket):
    usersWithoutID.remove(websocket)

# Entferne Nutzer mit ID
async def unregisterUserWithID(websocket, sessionID, controlStatus):
    tmpUser = user(sessionID, websocket, canControl)
    usersWithID.remove(tmpUser)

# Auswerten der empfangenen Nachricht
async def evaluateMessage(message, websocket):
    # Registrieren als neuer Nutzer mit ID
    if message["action"] == "register":
        registerUserWithID(websocket, message["sessionID"])
    elif message["action"] == "unregister":
        unregisterUserWithID(websocket, message["sessionID"], message["canControl"])
    elif message["action"] == 'setControl':
        changeControlStatus(message["sessionID"], websocket)
    elif message["action"] == 'input':
        sendNewInput(message["sessionID"], message["type"])
    else:
        logging.error("unsupported event: {}", message)
    
async def wsServer(websocket, path):
    await registerUninitUser(websocket)
    try:
        await websocket.send(state_event())
        async for message in websocket:
            data = json.loads(message)
            evaluateMessage(data, websocket)
    finally:
        await unregister(websocket)

In [3]:
start_server = websockets.serve(wsServer, "localhost", 6789)

asyncio.get_event_loop().run_until_complete(start_server)
asyncio.get_event_loop().run_forever()

RuntimeError: This event loop is already running